In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import polars as pl  # 导入 Polars
from pathlib import Path
from tqdm.auto import tqdm
import warnings
import os
import pickle # torch.load 内部需要 pickle

# --- 推理服务器 Imports ---
try:
    import kaggle_evaluation.nfl_inference_server
except ImportError:
    print("Warning: kaggle_evaluation not found. Server will not start in local test.")
    # 在本地测试时，定义一个模拟的 nfl_inference_server 以避免 NameError
    if 'kaggle_evaluation' not in globals():
        class MockNFLInferenceServer:
            def __init__(self, predict_fn):
                print("[MockServer] Initialized.")
            def serve(self):
                print("[MockServer] serve() called.")
            def run_local_gateway(self, data_path):
                print(f"[MockServer] run_local_gateway() called with {data_path}.")
        
        # 模拟 kaggle_evaluation 模块
        class MockKaggleEvaluation:
            class nfl_inference_server:
                NFLInferenceServer = MockNFLInferenceServer
        
        kaggle_evaluation = MockKaggleEvaluation()


from sklearn.preprocessing import StandardScaler
# 导入 StratifiedKFold 和 GroupKFold
from sklearn.model_selection import StratifiedKFold, GroupKFold 
from sklearn.cluster import KMeans
from multiprocessing import Pool as MultiprocessingPool, cpu_count

warnings.filterwarnings('ignore')

# ============================================================================
# CONFIG
# ============================================================================

class Config:
    # !!! 切换这里: True 用于训练, False 用于推理提交 !!!
    IS_TRAINING = False
    
    DATA_DIR = Path("/kaggle/input/nfl-big-data-bowl-2026-prediction")
    OUTPUT_DIR = Path("/kaggle/input/ubantu11/outputs") # 模型和缩放器将保存到这里
    OUTPUT_DIR.mkdir(exist_ok=True)
    
    SEED = 42
    N_FOLDS = 5
    BATCH_SIZE = 256
    EPOCHS = 200
    PATIENCE = 30
    LEARNING_RATE = 1e-3
    
    WINDOW_SIZE = 10
    HIDDEN_DIM = 128
    MAX_FUTURE_HORIZON = 94
    
    # === Transformer 超参数 ===
    N_HEADS = 4  # Transformer 的注意力头数
    N_LAYERS = 2 # Transformer 编码器的层数
    
    # === 新增：ResidualMLP Head 超参数 ===
    MLP_HIDDEN_DIM = 256 # MLP 头的内部隐藏维度
    N_RES_BLOCKS = 2     # 残差块的数量
    # ==================================
    
    FIELD_X_MIN, FIELD_X_MAX = 0.0, 120.0
    FIELD_Y_MIN, FIELD_Y_MAX = 0.0, 53.3
    
    K_NEIGH = 6
    RADIUS = 30.0
    TAU = 8.0
    N_ROUTE_CLUSTERS = 7
    
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def set_seed(seed=42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(Config.SEED)

# ============================================================================
# GEOMETRIC BASELINE - THE BREAKTHROUGH
# (此部分及以下特征工程函数未更改... 为简洁起见可折叠)
# ============================================================================

def compute_geometric_endpoint(df):
    """
    Compute where each player SHOULD end up based on geometry.
    This is the deterministic part - no learning needed.
    """
    df = df.copy()
    
    # Time to play end
    if 'num_frames_output' in df.columns:
        t_total = df['num_frames_output'] / 10.0
    else:
        t_total = 3.0
    
    df['time_to_endpoint'] = t_total
    
    # Initialize with momentum (default rule)
    df['geo_endpoint_x'] = df['x'] + df['velocity_x'] * t_total
    df['geo_endpoint_y'] = df['y'] + df['velocity_y'] * t_total
    
    # Rule 1: Targeted Receivers converge to ball
    if 'ball_land_x' in df.columns:
        receiver_mask = df['player_role'] == 'Targeted Receiver'
        df.loc[receiver_mask, 'geo_endpoint_x'] = df.loc[receiver_mask, 'ball_land_x']
        df.loc[receiver_mask, 'geo_endpoint_y'] = df.loc[receiver_mask, 'ball_land_y']
        
        # Rule 2: Defenders mirror receivers (maintain offset)
        defender_mask = df['player_role'] == 'Defensive Coverage'
        has_mirror = df.get('mirror_offset_x', 0).notna() & (df.get('mirror_wr_dist', 50) < 15)
        coverage_mask = defender_mask & has_mirror
        
        df.loc[coverage_mask, 'geo_endpoint_x'] = (
            df.loc[coverage_mask, 'ball_land_x'] + 
            df.loc[coverage_mask, 'mirror_offset_x'].fillna(0)
        )
        df.loc[coverage_mask, 'geo_endpoint_y'] = (
            df.loc[coverage_mask, 'ball_land_y'] + 
            df.loc[coverage_mask, 'mirror_offset_y'].fillna(0)
        )
    
    # Clip to field
    df['geo_endpoint_x'] = df['geo_endpoint_x'].clip(Config.FIELD_X_MIN, Config.FIELD_X_MAX)
    df['geo_endpoint_y'] = df['geo_endpoint_y'].clip(Config.FIELD_Y_MIN, Config.FIELD_Y_MAX)
    
    return df

def add_geometric_features(df):
    """Add features that describe the geometric solution"""
    df = compute_geometric_endpoint(df)
    
    # Vector to geometric endpoint
    df['geo_vector_x'] = df['geo_endpoint_x'] - df['x']
    df['geo_vector_y'] = df['geo_endpoint_y'] - df['y']
    df['geo_distance'] = np.sqrt(df['geo_vector_x']**2 + df['geo_vector_y']**2)
    
    # Required velocity to reach geometric endpoint
    t = df['time_to_endpoint'] + 0.1
    df['geo_required_vx'] = df['geo_vector_x'] / t
    df['geo_required_vy'] = df['geo_vector_y'] / t
    
    # Current velocity vs required
    df['geo_velocity_error_x'] = df['geo_required_vx'] - df['velocity_x']
    df['geo_velocity_error_y'] = df['geo_required_vy'] - df['velocity_y']
    df['geo_velocity_error'] = np.sqrt(
        df['geo_velocity_error_x']**2 + df['geo_velocity_error_y']**2
    )
    
    # Required constant acceleration (a = 2*Δx/t²)
    t_sq = t * t
    df['geo_required_ax'] = 2 * df['geo_vector_x'] / t_sq
    df['geo_required_ay'] = 2 * df['geo_vector_y'] / t_sq
    df['geo_required_ax'] = df['geo_required_ax'].clip(-10, 10)
    df['geo_required_ay'] = df['geo_required_ay'].clip(-10, 10)
    
    # Alignment with geometric path
    velocity_mag = np.sqrt(df['velocity_x']**2 + df['velocity_y']**2)
    geo_unit_x = df['geo_vector_x'] / (df['geo_distance'] + 0.1)
    geo_unit_y = df['geo_vector_y'] / (df['geo_distance'] + 0.1)
    df['geo_alignment'] = (
        df['velocity_x'] * geo_unit_x + df['velocity_y'] * geo_unit_y
    ) / (velocity_mag + 0.1)
    
    # Role-specific geometric quality
    df['geo_receiver_urgency'] = df['is_receiver'] * df['geo_distance'] / (t + 0.1)
    df['geo_defender_coupling'] = df['is_coverage'] * (1.0 / (df.get('mirror_wr_dist', 50) + 1.0))
    
    return df

# ============================================================================
# PROVEN FEATURE ENGINEERING (YOUR 0.59 BASE)
# (此部分未更改)
# ============================================================================

def get_velocity(speed, direction_deg):
    theta = np.deg2rad(direction_deg)
    return speed * np.sin(theta), speed * np.cos(theta)

def height_to_feet(height_str):
    try:
        ft, inches = map(int, str(height_str).split('-'))
        return ft + inches/12
    except:
        return 6.0

def get_opponent_features(input_df):
    """Enhanced opponent interaction with MIRROR WR tracking"""
    features = []
    # (在推理时关闭 tqdm)
    disable_tqdm = not Config.IS_TRAINING
    
    for (gid, pid), group in tqdm(input_df.groupby(['game_id', 'play_id']), 
                                  desc="🏈 Opponents", leave=False, disable=disable_tqdm):
        last = group.sort_values('frame_id').groupby('nfl_id').last()
        
        if len(last) < 2:
            continue
            
        positions = last[['x', 'y']].values
        sides = last['player_side'].values
        speeds = last['s'].values
        directions = last['dir'].values
        roles = last['player_role'].values
        
        receiver_mask = np.isin(roles, ['Targeted Receiver', 'Other Route Runner'])
        
        for i, (nid, side, role) in enumerate(zip(last.index, sides, roles)):
            opp_mask = sides != side
            
            feat = {
                'game_id': gid, 'play_id': pid, 'nfl_id': nid,
                'nearest_opp_dist': 50.0, 'closing_speed': 0.0,
                'num_nearby_opp_3': 0, 'num_nearby_opp_5': 0,
                'mirror_wr_vx': 0.0, 'mirror_wr_vy': 0.0,
                'mirror_offset_x': 0.0, 'mirror_offset_y': 0.0,
                'mirror_wr_dist': 50.0,
            }
            
            if not opp_mask.any():
                features.append(feat)
                continue
            
            opp_positions = positions[opp_mask]
            distances = np.sqrt(((positions[i] - opp_positions)**2).sum(axis=1))
            
            if len(distances) == 0:
                features.append(feat)
                continue
                
            nearest_idx = distances.argmin()
            feat['nearest_opp_dist'] = distances[nearest_idx]
            feat['num_nearby_opp_3'] = (distances < 3.0).sum()
            feat['num_nearby_opp_5'] = (distances < 5.0).sum()
            
            my_vx, my_vy = get_velocity(speeds[i], directions[i])
            opp_speeds = speeds[opp_mask]
            opp_dirs = directions[opp_mask]
            opp_vx, opp_vy = get_velocity(opp_speeds[nearest_idx], opp_dirs[nearest_idx])
            
            rel_vx = my_vx - opp_vx
            rel_vy = my_vy - opp_vy
            to_me = positions[i] - opp_positions[nearest_idx]
            to_me_norm = to_me / (np.linalg.norm(to_me) + 0.1)
            feat['closing_speed'] = -(rel_vx * to_me_norm[0] + rel_vy * to_me_norm[1])
            
            if role == 'Defensive Coverage' and receiver_mask.any():
                rec_positions = positions[receiver_mask]
                rec_distances = np.sqrt(((positions[i] - rec_positions)**2).sum(axis=1))
                
                if len(rec_distances) > 0:
                    closest_rec_idx = rec_distances.argmin()
                    rec_indices = np.where(receiver_mask)[0]
                    actual_rec_idx = rec_indices[closest_rec_idx]
                    
                    rec_vx, rec_vy = get_velocity(speeds[actual_rec_idx], directions[actual_rec_idx])
                    
                    feat['mirror_wr_vx'] = rec_vx
                    feat['mirror_wr_vy'] = rec_vy
                    feat['mirror_wr_dist'] = rec_distances[closest_rec_idx]
                    feat['mirror_offset_x'] = positions[i][0] - rec_positions[closest_rec_idx][0]
                    feat['mirror_offset_y'] = positions[i][1] - rec_positions[closest_rec_idx][1]
            
            features.append(feat)
    
    return pd.DataFrame(features)

def extract_route_patterns(input_df, kmeans=None, scaler=None, fit=True):
    """Route clustering"""
    route_features = []
    # (在推理时关闭 tqdm)
    disable_tqdm = not Config.IS_TRAINING
    
    for (gid, pid, nid), group in tqdm(input_df.groupby(['game_id', 'play_id', 'nfl_id']), 
                                      desc="🛣️  Routes", leave=False, disable=disable_tqdm):
        traj = group.sort_values('frame_id').tail(5)
        
        if len(traj) < 3:
            continue
        
        positions = traj[['x', 'y']].values
        speeds = traj['s'].values
        
        total_dist = np.sum(np.sqrt(np.diff(positions[:, 0])**2 + np.diff(positions[:, 1])**2))
        displacement = np.sqrt((positions[-1, 0] - positions[0, 0])**2 + 
                               (positions[-1, 1] - positions[0, 1])**2)
        straightness = displacement / (total_dist + 0.1)
        
        angles = np.arctan2(np.diff(positions[:, 1]), np.diff(positions[:, 0]))
        if len(angles) > 1:
            angle_changes = np.abs(np.diff(angles))
            max_turn = np.max(angle_changes)
            mean_turn = np.mean(angle_changes)
        else:
            max_turn = mean_turn = 0
        
        speed_mean = speeds.mean()
        speed_change = speeds[-1] - speeds[0] if len(speeds) > 1 else 0
        dx = positions[-1, 0] - positions[0, 0]
        dy = positions[-1, 1] - positions[0, 1]
        
        route_features.append({
            'game_id': gid, 'play_id': pid, 'nfl_id': nid,
            'traj_straightness': straightness,
            'traj_max_turn': max_turn,
            'traj_mean_turn': mean_turn,
            'traj_depth': abs(dx),
            'traj_width': abs(dy),
            'speed_mean': speed_mean,
            'speed_change': speed_change,
        })
    
    route_df = pd.DataFrame(route_features)
    if 'traj_straightness' not in route_df.columns:
        # 如果没有创建特征（例如，所有序列都太短）
        if fit:
            return pd.DataFrame(), KMeans(n_clusters=Config.N_ROUTE_CLUSTERS), StandardScaler()
        else:
            return pd.DataFrame()
            
    feat_cols = ['traj_straightness', 'traj_max_turn', 'traj_mean_turn',
                 'traj_depth', 'traj_width', 'speed_mean', 'speed_change']
    X = route_df[feat_cols].fillna(0)
    
    if fit:
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        kmeans = KMeans(n_clusters=Config.N_ROUTE_CLUSTERS, random_state=Config.SEED, n_init=10)
        route_df['route_pattern'] = kmeans.fit_predict(X_scaled)
        return route_df, kmeans, scaler
    else:
        # 在推理时，确保 kmeans 和 scaler 已经被加载
        if kmeans is None or scaler is None:
            raise ValueError("KMeans and Scaler must be provided during inference (fit=False)")
        X_scaled = scaler.transform(X)
        route_df['route_pattern'] = kmeans.predict(X_scaled)
        return route_df

def compute_neighbor_embeddings(input_df, k_neigh=Config.K_NEIGH, 
                                radius=Config.RADIUS, tau=Config.TAU):
    """GNN-lite embeddings"""
    # (在推理时关闭打印)
    if Config.IS_TRAINING:
        print("🕸️  GNN embeddings...")
    
    cols_needed = ["game_id", "play_id", "nfl_id", "frame_id", "x", "y", 
                   "velocity_x", "velocity_y", "player_side"]
    src = input_df[cols_needed].copy()
    
    last = (src.sort_values(["game_id", "play_id", "nfl_id", "frame_id"])
               .groupby(["game_id", "play_id", "nfl_id"], as_index=False)
               .tail(1)
               .rename(columns={"frame_id": "last_frame_id"})
               .reset_index(drop=True))
    
    tmp = last.merge(
        src.rename(columns={
            "frame_id": "nb_frame_id", "nfl_id": "nfl_id_nb",
            "x": "x_nb", "y": "y_nb", 
            "velocity_x": "vx_nb", "velocity_y": "vy_nb", 
            "player_side": "player_side_nb"
        }),
        left_on=["game_id", "play_id", "last_frame_id"],
        right_on=["game_id", "play_id", "nb_frame_id"],
        how="left"
    )
    
    tmp = tmp[tmp["nfl_id_nb"] != tmp["nfl_id"]]
    tmp["dx"] = tmp["x_nb"] - tmp["x"]
    tmp["dy"] = tmp["y_nb"] - tmp["y"]
    tmp["dvx"] = tmp["vx_nb"] - tmp["velocity_x"]
    tmp["dvy"] = tmp["vy_nb"] - tmp["velocity_y"]
    tmp["dist"] = np.sqrt(tmp["dx"]**2 + tmp["dy"]**2)
    
    tmp = tmp[np.isfinite(tmp["dist"]) & (tmp["dist"] > 1e-6)]
    if radius is not None:
        tmp = tmp[tmp["dist"] <= radius]
    
    tmp["is_ally"] = (tmp["player_side_nb"] == tmp["player_side"]).astype(np.float32)
    
    keys = ["game_id", "play_id", "nfl_id"]
    tmp["rnk"] = tmp.groupby(keys)["dist"].rank(method="first")
    if k_neigh is not None:
        tmp = tmp[tmp["rnk"] <= float(k_neigh)]
    
    tmp["w"] = np.exp(-tmp["dist"] / float(tau))
    sum_w = tmp.groupby(keys)["w"].transform("sum")
    tmp["wn"] = np.where(sum_w > 0, tmp["w"] / sum_w, 0.0)
    
    tmp["wn_ally"] = tmp["wn"] * tmp["is_ally"]
    tmp["wn_opp"] = tmp["wn"] * (1.0 - tmp["is_ally"])
    
    for col in ["dx", "dy", "dvx", "dvy"]:
        tmp[f"{col}_ally_w"] = tmp[col] * tmp["wn_ally"]
        tmp[f"{col}_opp_w"] = tmp[col] * tmp["wn_opp"]
    
    tmp["dist_ally"] = np.where(tmp["is_ally"] > 0.5, tmp["dist"], np.nan)
    tmp["dist_opp"] = np.where(tmp["is_ally"] < 0.5, tmp["dist"], np.nan)
    
    ag = tmp.groupby(keys).agg(
        gnn_ally_dx_mean=("dx_ally_w", "sum"),
        gnn_ally_dy_mean=("dy_ally_w", "sum"),
        gnn_ally_dvx_mean=("dvx_ally_w", "sum"),
        gnn_ally_dvy_mean=("dvy_ally_w", "sum"),
        gnn_opp_dx_mean=("dx_opp_w", "sum"),
        gnn_opp_dy_mean=("dy_opp_w", "sum"),
        gnn_opp_dvx_mean=("dvx_opp_w", "sum"),
        gnn_opp_dvy_mean=("dvy_opp_w", "sum"),
        gnn_ally_cnt=("is_ally", "sum"),
        gnn_opp_cnt=("is_ally", lambda s: float(len(s) - s.sum())),
        gnn_ally_dmin=("dist_ally", "min"),
        gnn_ally_dmean=("dist_ally", "mean"),
        gnn_opp_dmin=("dist_opp", "min"),
        gnn_opp_dmean=("dist_opp", "mean"),
    ).reset_index()
    
    near = tmp.loc[tmp["rnk"] <= 3, keys + ["rnk", "dist"]].copy()
    if len(near) > 0:
        near["rnk"] = near["rnk"].astype(int)
        dwide = near.pivot_table(index=keys, columns="rnk", values="dist", aggfunc="first")
        dwide = dwide.rename(columns={1: "gnn_d1", 2: "gnn_d2", 3: "gnn_d3"}).reset_index()
        ag = ag.merge(dwide, on=keys, how="left")
    
    for c in ["gnn_ally_dx_mean", "gnn_ally_dy_mean", "gnn_ally_dvx_mean", "gnn_ally_dvy_mean",
              "gnn_opp_dx_mean", "gnn_opp_dy_mean", "gnn_opp_dvx_mean", "gnn_opp_dvy_mean"]:
        ag[c] = ag[c].fillna(0.0)
    for c in ["gnn_ally_cnt", "gnn_opp_cnt"]:
        ag[c] = ag[c].fillna(0.0)
    for c in ["gnn_ally_dmin", "gnn_opp_dmin", "gnn_ally_dmean", "gnn_opp_dmean", 
              "gnn_d1", "gnn_d2", "gnn_d3"]:
        ag[c] = ag[c].fillna(radius if radius is not None else 30.0)
    
    return ag

# ============================================================================
# SEQUENCE PREPARATION WITH GEOMETRIC FEATURES
# (此部分未更改)
# ============================================================================

def prepare_sequences_geometric(input_df, output_df=None, test_template=None, 
                                is_training=True, window_size=10,
                                route_kmeans=None, route_scaler=None):
    """YOUR 154 features + 13 geometric features = 167 total"""
    
    # 在推理时，关闭 tqdm 进度条，因为它在服务器日志中很吵
    disable_tqdm = not is_training
    
    if is_training:
        print(f"\n{'='*80}")
        print(f"PREPARING GEOMETRIC SEQUENCES (is_training={is_training})")
        print(f"{'='*80}")
    
    input_df = input_df.copy()
    input_df = input_df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
    
    if not disable_tqdm: print("Step 1: Base features...")
    
    input_df['player_height_feet'] = input_df['player_height'].apply(height_to_feet)
    height_parts = input_df['player_height'].str.split('-', expand=True)
    input_df['height_inches'] = height_parts[0].astype(float) * 12 + height_parts[1].astype(float)
    input_df['bmi'] = (input_df['player_weight'] / (input_df['height_inches']**2)) * 703
    
    dir_rad = np.deg2rad(input_df['dir'].fillna(0))
    input_df['velocity_x'] = input_df['s'] * np.sin(dir_rad)
    input_df['velocity_y'] = input_df['s'] * np.cos(dir_rad)
    input_df['acceleration_x'] = input_df['a'] * np.cos(dir_rad)
    input_df['acceleration_y'] = input_df['a'] * np.sin(dir_rad)
    
    input_df['speed_squared'] = input_df['s'] ** 2
    input_df['accel_magnitude'] = np.sqrt(input_df['acceleration_x']**2 + input_df['acceleration_y']**2)
    input_df['momentum_x'] = input_df['velocity_x'] * input_df['player_weight']
    input_df['momentum_y'] = input_df['velocity_y'] * input_df['player_weight']
    input_df['kinetic_energy'] = 0.5 * input_df['player_weight'] * input_df['speed_squared']
    
    input_df['orientation_diff'] = np.abs(input_df['o'] - input_df['dir'])
    input_df['orientation_diff'] = np.minimum(input_df['orientation_diff'], 360 - input_df['orientation_diff'])
    
    input_df['is_offense'] = (input_df['player_side'] == 'Offense').astype(int)
    input_df['is_defense'] = (input_df['player_side'] == 'Defense').astype(int)
    input_df['is_receiver'] = (input_df['player_role'] == 'Targeted Receiver').astype(int)
    input_df['is_coverage'] = (input_df['player_role'] == 'Defensive Coverage').astype(int)
    input_df['is_passer'] = (input_df['player_role'] == 'Passer').astype(int)
    input_df['role_targeted_receiver'] = input_df['is_receiver']
    input_df['role_defensive_coverage'] = input_df['is_coverage']
    input_df['role_passer'] = input_df['is_passer']
    input_df['side_offense'] = input_df['is_offense']
    
    if 'ball_land_x' in input_df.columns:
        ball_dx = input_df['ball_land_x'] - input_df['x']
        ball_dy = input_df['ball_land_y'] - input_df['y']
        input_df['distance_to_ball'] = np.sqrt(ball_dx**2 + ball_dy**2)
        input_df['dist_to_ball'] = input_df['distance_to_ball']
        input_df['dist_squared'] = input_df['distance_to_ball'] ** 2
        input_df['angle_to_ball'] = np.arctan2(ball_dy, ball_dx)
        input_df['ball_direction_x'] = ball_dx / (input_df['distance_to_ball'] + 1e-6)
        input_df['ball_direction_y'] = ball_dy / (input_df['distance_to_ball'] + 1e-6)
        input_df['closing_speed_ball'] = (
            input_df['velocity_x'] * input_df['ball_direction_x'] +
            input_df['velocity_y'] * input_df['ball_direction_y']
        )
        input_df['velocity_toward_ball'] = (
            input_df['velocity_x'] * np.cos(input_df['angle_to_ball']) + 
            input_df['velocity_y'] * np.sin(input_df['angle_to_ball'])
        )
        input_df['velocity_alignment'] = np.cos(input_df['angle_to_ball'] - dir_rad)
        input_df['angle_diff'] = np.abs(input_df['o'] - np.degrees(input_df['angle_to_ball']))
        input_df['angle_diff'] = np.minimum(input_df['angle_diff'], 360 - input_df['angle_diff'])
    
    if not disable_tqdm: print("Step 2: Advanced features...")
    
    opp_features = get_opponent_features(input_df) # TQDM 在函数内部
    input_df = input_df.merge(opp_features, on=['game_id', 'play_id', 'nfl_id'], how='left')
    
    if is_training:
        route_features, route_kmeans, route_scaler = extract_route_patterns(input_df, fit=True)
    else:
        # 在推理时，route_kmeans 和 route_scaler 必须被传入
        route_features = extract_route_patterns(input_df, route_kmeans, route_scaler, fit=False)
    
    if not route_features.empty:
        input_df = input_df.merge(route_features, on=['game_id', 'play_id', 'nfl_id'], how='left')
    
    gnn_features = compute_neighbor_embeddings(input_df) # 打印在函数内部
    input_df = input_df.merge(gnn_features, on=['game_id', 'play_id', 'nfl_id'], how='left')
    
    if 'nearest_opp_dist' in input_df.columns:
        input_df['pressure'] = 1 / np.maximum(input_df['nearest_opp_dist'], 0.5)
        input_df['under_pressure'] = (input_df['nearest_opp_dist'] < 3).astype(int)
        input_df['pressure_x_speed'] = input_df['pressure'] * input_df['s']
    
    if 'mirror_wr_vx' in input_df.columns:
        s_safe = np.maximum(input_df['s'], 0.1)
        input_df['mirror_similarity'] = (
            input_df['velocity_x'] * input_df['mirror_wr_vx'] + 
            input_df['velocity_y'] * input_df['mirror_wr_vy']
        ) / s_safe
        input_df['mirror_offset_dist'] = np.sqrt(
            input_df['mirror_offset_x']**2 + input_df['mirror_offset_y']**2
        )
        input_df['mirror_alignment'] = input_df['mirror_similarity'] * input_df['role_defensive_coverage']
    
    if not disable_tqdm: print("Step 3: Temporal features...")
    
    gcols = ['game_id', 'play_id', 'nfl_id']
    
    for lag in [1, 2, 3, 4, 5]:
        for col in ['x', 'y', 'velocity_x', 'velocity_y', 's', 'a']:
            if col in input_df.columns:
                input_df[f'{col}_lag{lag}'] = input_df.groupby(gcols)[col].shift(lag)
    
    for window in [3, 5]:
        for col in ['x', 'y', 'velocity_x', 'velocity_y', 's']:
            if col in input_df.columns:
                input_df[f'{col}_rolling_mean_{window}'] = (
                    input_df.groupby(gcols)[col]
                            .rolling(window, min_periods=1).mean()
                            .reset_index(level=[0,1,2], drop=True)
                )
                input_df[f'{col}_rolling_std_{window}'] = (
                    input_df.groupby(gcols)[col]
                            .rolling(window, min_periods=1).std()
                            .reset_index(level=[0,1,2], drop=True)
                )
    
    for col in ['velocity_x', 'velocity_y']:
        if col in input_df.columns:
            input_df[f'{col}_delta'] = input_df.groupby(gcols)[col].diff()
    
    input_df['velocity_x_ema'] = input_df.groupby(gcols)['velocity_x'].transform(
        lambda x: x.ewm(alpha=0.3, adjust=False).mean()
    )
    input_df['velocity_y_ema'] = input_df.groupby(gcols)['velocity_y'].transform(
        lambda x: x.ewm(alpha=0.3, adjust=False).mean()
    )
    input_df['speed_ema'] = input_df.groupby(gcols)['s'].transform(
        lambda x: x.ewm(alpha=0.3, adjust=False).mean()
    )
    
    if not disable_tqdm: print("Step 4: Time features...")
    
    if 'num_frames_output' in input_df.columns:
        max_frames = input_df['num_frames_output']
        
        input_df['max_play_duration'] = max_frames / 10.0
        input_df['frame_time'] = input_df['frame_id'] / 10.0
        input_df['progress_ratio'] = input_df['frame_id'] / np.maximum(max_frames, 1)
        input_df['time_remaining'] = (max_frames - input_df['frame_id']) / 10.0
        input_df['frames_remaining'] = max_frames - input_df['frame_id']
        
        input_df['expected_x_at_ball'] = input_df['x'] + input_df['velocity_x'] * input_df['frame_time']
        input_df['expected_y_at_ball'] = input_df['y'] + input_df['velocity_y'] * input_df['frame_time']
        
        if 'ball_land_x' in input_df.columns:
            input_df['error_from_ball_x'] = input_df['expected_x_at_ball'] - input_df['ball_land_x']
            input_df['error_from_ball_y'] = input_df['expected_y_at_ball'] - input_df['ball_land_y']
            input_df['error_from_ball'] = np.sqrt(
                input_df['error_from_ball_x']**2 + input_df['error_from_ball_y']**2
            )
            
            input_df['weighted_dist_by_time'] = input_df['dist_to_ball'] / (input_df['frame_time'] + 0.1)
            input_df['dist_scaled_by_progress'] = input_df['dist_to_ball'] * (1 - input_df['progress_ratio'])
        
        input_df['time_squared'] = input_df['frame_time'] ** 2
        input_df['velocity_x_progress'] = input_df['velocity_x'] * input_df['progress_ratio']
        input_df['velocity_y_progress'] = input_df['velocity_y'] * input_df['progress_ratio']
        input_df['speed_scaled_by_time_left'] = input_df['s'] * input_df['time_remaining']
        
        input_df['actual_play_length'] = max_frames
        input_df['length_ratio'] = max_frames / 30.0
    
    # 🎯 THE BREAKTHROUGH: Add geometric features
    if not disable_tqdm: print("Step 5: 🎯 Geometric endpoint features...")
    input_df = add_geometric_features(input_df)
    
    if not disable_tqdm: print("Step 6: Building feature list...")
    
    # 你的 154 个成熟特征
    feature_cols = [
        'x', 'y', 's', 'a', 'o', 'dir', 'frame_id', 'ball_land_x', 'ball_land_y',
        'player_height_feet', 'player_weight', 'height_inches', 'bmi',
        'velocity_x', 'velocity_y', 'acceleration_x', 'acceleration_y',
        'momentum_x', 'momentum_y', 'kinetic_energy',
        'speed_squared', 'accel_magnitude', 'orientation_diff',
        'is_offense', 'is_defense', 'is_receiver', 'is_coverage', 'is_passer',
        'role_targeted_receiver', 'role_defensive_coverage', 'role_passer', 'side_offense',
        'distance_to_ball', 'dist_to_ball', 'dist_squared', 'angle_to_ball', 
        'ball_direction_x', 'ball_direction_y', 'closing_speed_ball',
        'velocity_toward_ball', 'velocity_alignment', 'angle_diff',
        'nearest_opp_dist', 'closing_speed', 'num_nearby_opp_3', 'num_nearby_opp_5',
        'mirror_wr_vx', 'mirror_wr_vy', 'mirror_offset_x', 'mirror_offset_y',
        'pressure', 'under_pressure', 'pressure_x_speed', 
        'mirror_similarity', 'mirror_offset_dist', 'mirror_alignment',
        'route_pattern', 'traj_straightness', 'traj_max_turn', 'traj_mean_turn',
        'traj_depth', 'traj_width', 'speed_mean', 'speed_change',
        'gnn_ally_dx_mean', 'gnn_ally_dy_mean', 'gnn_ally_dvx_mean', 'gnn_ally_dvy_mean',
        'gnn_opp_dx_mean', 'gnn_opp_dy_mean', 'gnn_opp_dvx_mean', 'gnn_opp_dvy_mean',
        'gnn_ally_cnt', 'gnn_opp_cnt',
        'gnn_ally_dmin', 'gnn_ally_dmean', 'gnn_opp_dmin', 'gnn_opp_dmean',
        'gnn_d1', 'gnn_d2', 'gnn_d3',
    ]
    
    for lag in [1, 2, 3, 4, 5]:
        for col in ['x', 'y', 'velocity_x', 'velocity_y', 's', 'a']:
            feature_cols.append(f'{col}_lag{lag}')
    
    for window in [3, 5]:
        for col in ['x', 'y', 'velocity_x', 'velocity_y', 's']:
            feature_cols.append(f'{col}_rolling_mean_{window}')
            feature_cols.append(f'{col}_rolling_std_{window}')
    
    feature_cols.extend(['velocity_x_delta', 'velocity_y_delta'])
    feature_cols.extend(['velocity_x_ema', 'velocity_y_ema', 'speed_ema'])
    
    feature_cols.extend([
        'max_play_duration', 'frame_time', 'progress_ratio', 'time_remaining', 'frames_remaining',
        'expected_x_at_ball', 'expected_y_at_ball', 
        'error_from_ball_x', 'error_from_ball_y', 'error_from_ball',
        'time_squared', 'weighted_dist_by_time', 
        'velocity_x_progress', 'velocity_y_progress', 'dist_scaled_by_progress',
        'speed_scaled_by_time_left', 'actual_play_length', 'length_ratio',
    ])
    
    # 🎯 Add 13 geometric features
    feature_cols.extend([
        'geo_endpoint_x', 'geo_endpoint_y',
        'geo_vector_x', 'geo_vector_y', 'geo_distance',
        'geo_required_vx', 'geo_required_vy',
        'geo_velocity_error_x', 'geo_velocity_error_y', 'geo_velocity_error',
        'geo_required_ax', 'geo_required_ay',
        'geo_alignment',
    ])
    
    # 确保特征存在
    feature_cols = [c for c in feature_cols if c in input_df.columns]
    
    if not disable_tqdm: print(f"✓ Using {len(feature_cols)} features")
    
    if not disable_tqdm: print("Step 7: Creating sequences...")
    
    input_df.set_index(['game_id', 'play_id', 'nfl_id'], inplace=True)
    grouped = input_df.groupby(level=['game_id', 'play_id', 'nfl_id'])
    
    target_rows = output_df if is_training else test_template
    target_groups = target_rows[['game_id', 'play_id', 'nfl_id']].drop_duplicates()
    
    sequences, targets_dx, targets_dy, targets_frame_ids, sequence_ids = [], [], [], [], []
    geo_endpoints_x, geo_endpoints_y = [], []
    
    for _, row in tqdm(target_groups.iterrows(), total=len(target_groups), desc="Creating sequences", disable=disable_tqdm):
        key = (row['game_id'], row['play_id'], row['nfl_id'])
        
        try:
            group_df = grouped.get_group(key)
        except KeyError:
            continue
        
        input_window = group_df.tail(window_size)
        
        if len(input_window) < window_size:
            # 仅在训练时跳过。在测试时，我们需要填充以进行预测。
            if is_training:
                continue
            pad_len = window_size - len(input_window)
            pad_df = pd.DataFrame(np.nan, index=range(pad_len), columns=input_window.columns)
            input_window = pd.concat([pad_df, input_window], ignore_index=True)
        
        # 填充均值
        input_window = input_window.fillna(group_df.mean(numeric_only=True))
        seq = input_window[feature_cols].values
        
        if np.isnan(seq).any():
            # 再次检查，如果仍然有 NaN（例如，如果 group_df 为空或全为 NaN）
            if is_training:
                continue
            seq = np.nan_to_num(seq, nan=0.0) # 在推理时填充 0
        
        sequences.append(seq)
        
        # Store geometric endpoint for this player
        geo_x = input_window.iloc[-1]['geo_endpoint_x']
        geo_y = input_window.iloc[-1]['geo_endpoint_y']
        geo_endpoints_x.append(geo_x)
        geo_endpoints_y.append(geo_y)
        
        if is_training:
            out_grp = output_df[
                (output_df['game_id']==row['game_id']) &
                (output_df['play_id']==row['play_id']) &
                (output_df['nfl_id']==row['nfl_id'])
            ].sort_values('frame_id')
            
            last_x = input_window.iloc[-1]['x']
            last_y = input_window.iloc[-1]['y']
            
            dx = out_grp['x'].values - last_x
            dy = out_grp['y'].values - last_y
            
            targets_dx.append(dx)
            targets_dy.append(dy)
            targets_frame_ids.append(out_grp['frame_id'].values)
        
        sequence_ids.append({
            'game_id': key[0],
            'play_id': key[1],
            'nfl_id': key[2],
            'frame_id': input_window.iloc[-1]['frame_id']
        })
    
    if not disable_tqdm: print(f"✓ Created {len(sequences)} sequences")
    
    if is_training:
        return (sequences, targets_dx, targets_dy, targets_frame_ids, sequence_ids, 
                geo_endpoints_x, geo_endpoints_y, route_kmeans, route_scaler, feature_cols)
    
    # 在推理模式下
    return sequences, sequence_ids, geo_endpoints_x, geo_endpoints_y, feature_cols

# ============================================================================
# MODEL ARCHITECTURE (ST-TRANSFORMER with ResidualMLP Head)
# ============================================================================

class ResidualBlock(nn.Module):
    """
    一个标准的残差块：FFN + 快捷连接
    """
    def __init__(self, dim, hidden_dim, dropout=0.1):
        super().__init__()
        self.ffn = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
        self.norm = nn.LayerNorm(dim)
    
    def forward(self, x):
        # Pre-normalization
        return x + self.ffn(self.norm(x))

class ResidualMLPHead(nn.Module):
    """
    替换原有的 nn.Sequential Head
    """
    def __init__(self, input_dim, hidden_dim, output_dim, n_res_blocks=2, dropout=0.2):
        super().__init__()
        # 1. 从 context_dim (128) 投影到 mlp_hidden_dim (256)
        self.input_layer = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU()
        )
        
        # 2. 一系列的残差块 (在 256 维度上操作)
        self.residual_blocks = nn.Sequential(
            *[ResidualBlock(hidden_dim, hidden_dim * 2, dropout) for _ in range(n_res_blocks)]
        )
        
        # 3. 最后的 LayerNorm 和输出投影
        self.output_norm = nn.LayerNorm(hidden_dim)
        self.output_layer = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        x = self.input_layer(x)
        x = self.residual_blocks(x)
        x = self.output_norm(x)
        x = self.output_layer(x)
        return x

class STTransformer(nn.Module):
    """
    Spatio-Temporal Transformer
    """
    def __init__(self, input_dim, hidden_dim, horizon, window_size, n_heads, n_layers, dropout=0.1):
        super().__init__()
        config = Config() # 获取 MLP 的超参数
        self.horizon = horizon
        self.hidden_dim = hidden_dim

        # 1. Spatio: 特征嵌入
        self.input_projection = nn.Linear(input_dim, hidden_dim)
        
        # 2. Temporal: 可学习的位置编码
        self.pos_embed = nn.Parameter(torch.randn(1, window_size, hidden_dim)) 
        self.embed_dropout = nn.Dropout(dropout)

        # 3. Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim,
            nhead=n_heads,
            dim_feedforward=hidden_dim * 4,
            dropout=dropout,
            batch_first=True,
            activation='gelu'
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=n_layers
        )

        # 4. 池化 (复用你成熟的 Attention Pooling 机制)
        self.pool_ln = nn.LayerNorm(hidden_dim)
        self.pool_attn = nn.MultiheadAttention(hidden_dim, num_heads=n_heads, batch_first=True)
        self.pool_query = nn.Parameter(torch.randn(1, 1, hidden_dim)) 

        # 5. 输出 Head (!!! 已替换为 ResidualMLPHead !!!)
        self.head = ResidualMLPHead(
            input_dim=hidden_dim,                   # 128
            hidden_dim=config.MLP_HIDDEN_DIM,       # 256
            output_dim=horizon * 2,                 # 188
            n_res_blocks=config.N_RES_BLOCKS,       # 2
            dropout=0.2
        )

    def forward(self, x):
        B, S, _ = x.shape
        x_embed = self.input_projection(x) 
        x = x_embed + self.pos_embed[:, :S, :] 
        x = self.embed_dropout(x)
        
        h = self.transformer_encoder(x) 

        q = self.pool_query.expand(B, -1, -1)
        ctx, _ = self.pool_attn(q, self.pool_ln(h), self.pool_ln(h))
        ctx = ctx.squeeze(1) 

        out = self.head(ctx) # <--- 使用新的 Head
        out = out.view(B, self.horizon, 2)
        
        out = torch.cumsum(out, dim=1)
        
        return out

# ============================================================================
# LOSS (YOUR PROVEN TEMPORAL HUBER)
# (此部分未更改)
# ============================================================================

class TemporalHuber(nn.Module):
    def __init__(self, delta=0.5, time_decay=0.03):
        super().__init__()
        self.delta = delta
        self.time_decay = time_decay
    
    def forward(self, pred, target, mask):
        err = pred - target
        abs_err = torch.abs(err)
        huber = torch.where(abs_err <= self.delta, 0.5 * err * err, 
                            self.delta * (abs_err - 0.5 * self.delta))
        
        if self.time_decay > 0:
            L = pred.size(1)
            t = torch.arange(L, device=pred.device).float()
            weight = torch.exp(-self.time_decay * t).view(1, L, 1)
            huber = huber * weight
            mask = mask.unsqueeze(-1) * weight
        
        return (huber * mask).sum() / (mask.sum() + 1e-8)

# ============================================================================
# TRAINING
# (此部分未更改)
# ============================================================================

def prepare_targets(batch_dx, batch_dy, max_h):
    tensors_x, tensors_y, masks = [], [], []
    
    for dx, dy in zip(batch_dx, batch_dy):
        L = len(dx)
        padded_x = np.pad(dx, (0, max_h - L), constant_values=0).astype(np.float32)
        padded_y = np.pad(dy, (0, max_h - L), constant_values=0).astype(np.float32)
        mask = np.zeros(max_h, dtype=np.float32)
        mask[:L] = 1.0
        
        tensors_x.append(torch.tensor(padded_x))
        tensors_y.append(torch.tensor(padded_y))
        masks.append(torch.tensor(mask))
    
    targets = torch.stack([torch.stack(tensors_x), torch.stack(tensors_y)], dim=-1)
    return targets, torch.stack(masks)

def train_model(X_train, y_train_dx, y_train_dy, X_val, y_val_dx, y_val_dy, 
                input_dim, horizon, config):
    device = config.DEVICE
    
    # (模型实例化会自动使用新的 ResidualMLPHead)
    model = STTransformer(
        input_dim=input_dim,
        hidden_dim=config.HIDDEN_DIM,
        horizon=horizon,
        window_size=config.WINDOW_SIZE,
        n_heads=config.N_HEADS,
        n_layers=config.N_LAYERS
    ).to(device)
    
    criterion = TemporalHuber(delta=0.5, time_decay=0.03)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.LEARNING_RATE, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)
    
    train_batches = []
    for i in range(0, len(X_train), config.BATCH_SIZE):
        end = min(i + config.BATCH_SIZE, len(X_train))
        bx = torch.tensor(np.stack(X_train[i:end]).astype(np.float32))
        by, bm = prepare_targets(
            [y_train_dx[j] for j in range(i, end)],
            [y_train_dy[j] for j in range(i, end)],
            horizon
        )
        train_batches.append((bx, by, bm))
    
    val_batches = []
    for i in range(0, len(X_val), config.BATCH_SIZE):
        end = min(i + config.BATCH_SIZE, len(X_val))
        bx = torch.tensor(np.stack(X_val[i:end]).astype(np.float32))
        by, bm = prepare_targets(
            [y_val_dx[j] for j in range(i, end)],
            [y_val_dy[j] for j in range(i, end)],
            horizon
        )
        val_batches.append((bx, by, bm))
    
    best_loss, best_state, bad = float('inf'), None, 0
    
    for epoch in range(1, config.EPOCHS + 1):
        model.train()
        train_losses = []
        for bx, by, bm in train_batches:
            bx, by, bm = bx.to(device), by.to(device), bm.to(device)
            pred = model(bx)
            loss = criterion(pred, by, bm)
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            train_losses.append(loss.item())
        
        model.eval()
        val_losses = []
        with torch.no_grad():
            for bx, by, bm in val_batches:
                bx, by, bm = bx.to(device), by.to(device), bm.to(device)
                pred = model(bx)
                val_losses.append(criterion(pred, by, bm).item())
        
        train_loss, val_loss = np.mean(train_losses), np.mean(val_losses)
        scheduler.step(val_loss)
        
        if epoch % 10 == 0:
            print(f"  Epoch {epoch}: train={train_loss:.4f}, val={val_loss:.4f}")
        
        if val_loss < best_loss:
            best_loss = val_loss
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            bad = 0
        else:
            bad += 1
            if bad >= config.PATIENCE:
                print(f"  Early stop at epoch {epoch}")
                break
    
    if best_state:
        model.load_state_dict(best_state)
    
    return model, best_loss

# ============================================================================
# MAIN (已修改为使用 GroupKFold)
# ============================================================================

def main():
    config = Config()
    
    # 这个函数现在只在 IS_TRAINING = True 时被调用
    print("🏆  NFL BDB 2026 - GEOMETRIC ST-TRANSFORMER (TRAINING) 🏆")
    
    # [1/4] 加载训练数据
    print("\n[1/4] Loading training data...")
    train_input_files = [config.DATA_DIR / f"train/input_2023_w{w:02d}.csv" for w in range(1, 19)]
    train_output_files = [config.DATA_DIR / f"train/output_2023_w{w:02d}.csv" for w in range(1, 19)]
    train_input = pd.concat([pd.read_csv(f) for f in train_input_files if f.exists()])
    train_output = pd.concat([pd.read_csv(f) for f in train_output_files if f.exists()])
    print(f"✓ Train input: {train_input.shape}, Train output: {train_output.shape}")

    # [2/4] 准备训练序列
    print("\n[2/4] Preparing geometric sequences...")
    result = prepare_sequences_geometric(
        train_input, train_output, is_training=True, window_size=config.WINDOW_SIZE
    )
    # 注意：feature_cols 列表现在也被返回
    sequences, targets_dx, targets_dy, targets_frame_ids, sequence_ids, geo_x, geo_y, route_kmeans, route_scaler, feature_cols = result
    
    # 获取 input_dim 以便保存
    input_dim = sequences[0].shape[1]
    
    sequences = list(sequences)
    targets_dx = list(targets_dx)
    targets_dy = list(targets_dy)
    
    # --- 保存特征工程对象 (使用 .pth) ---
    torch.save(route_kmeans, config.OUTPUT_DIR / "route_kmeans.pth")
    torch.save(route_scaler, config.OUTPUT_DIR / "route_scaler.pth")
    print(f"✓ Saved route K-Means and scaler to {config.OUTPUT_DIR} (as .pth)")

    # [3/4] 训练模型 (!!! 已更改为 GroupKFold !!!)
    print("\n[3/4] Training geometric models with GroupKFold...")
    
    # --- GroupKFold 逻辑 ---
    # 我们需要一个唯一的 ID 来代表每个 "play"
    # 'play_id' 本身可能不是全局唯一的（例如，不同 game 之间）
    groups_raw = [f"{sid['game_id']}_{sid['play_id']}" for sid in sequence_ids]
    # 将字符串组 ID 转换为整数
    unique_groups, groups = np.unique(groups_raw, return_inverse=True)
    print(f"✓ Created {len(unique_groups)} unique groups (game_play_id) for GroupKFold.")
    
    gkf = GroupKFold(n_splits=config.N_FOLDS)
    # -------------------------
    
    oof_predictions = {}
    fold_rmses = []
    
    # (y=None, groups=groups)
    for fold, (tr, va) in enumerate(gkf.split(sequences, y=None, groups=groups), 1):
        print(f"\n{'='*60}")
        print(f"Fold {fold}/{config.N_FOLDS}")
        print(f"{'='*60}")
        
        X_tr = [sequences[i] for i in tr]
        X_va = [sequences[i] for i in va]
        y_tr_dx = [targets_dx[i] for i in tr]
        y_va_dx = [targets_dx[i] for i in va]
        y_tr_dy = [targets_dy[i] for i in tr]
        y_va_dy = [targets_dy[i] for i in va]
        
        scaler = StandardScaler()
        scaler.fit(np.vstack([s for s in X_tr]))
        
        X_tr_sc = [scaler.transform(s) for s in X_tr]
        X_va_sc = [scaler.transform(s) for s in X_va]
        
        model, loss = train_model(
            X_tr_sc, y_tr_dx, y_tr_dy,
            X_va_sc, y_va_dx, y_va_dy,
            input_dim, config.MAX_FUTURE_HORIZON, config # 使用正确的 input_dim
        )
        
        # --- 保存模型和缩放器 (使用 .pth) ---
        model_path = config.OUTPUT_DIR / f"st_transformer_fold_{fold}.pth"
        scaler_path = config.OUTPUT_DIR / f"scaler_fold_{fold}.pth"
        
        torch.save(model.state_dict(), model_path)
        torch.save(scaler, scaler_path)
        
        print(f"✓ Saved model to {model_path}")
        print(f"✓ Saved scaler to {scaler_path}")
        
        # --- OOF 预测 (用于 CV 验证) ---
        print(f"  Generating OOF predictions for fold {fold}...")
        model.eval()
        oof_preds_fold = []
        with torch.no_grad():
            for i in range(0, len(X_va_sc), config.BATCH_SIZE):
                end = min(i + config.BATCH_SIZE, len(X_va_sc))
                batch_x = torch.tensor(np.stack(X_va_sc[i:end]).astype(np.float32)).to(config.DEVICE)
                batch_preds = model(batch_x).cpu().numpy()
                oof_preds_fold.append(batch_preds)
        
        oof_preds_fold = np.vstack(oof_preds_fold)

        pred_coords, true_coords = [], []
        for i in range(len(oof_preds_fold)):
            original_idx = va[i]
            seq_id_info = sequence_ids[original_idx]
            sample_id = f"{seq_id_info['game_id']}_{seq_id_info['play_id']}_{seq_id_info['nfl_id']}"
            
            true_len = len(y_va_dx[i])
            pred_dx = oof_preds_fold[i, :true_len, 0]
            pred_dy = oof_preds_fold[i, :true_len, 1]
            
            oof_predictions[sample_id] = {
                'pred_dx': pred_dx, 'pred_dy': pred_dy,
                'true_dx': y_va_dx[i], 'true_dy': y_va_dy[i]
            }
            
            pred_coords.extend(list(zip(pred_dx, pred_dy)))
            true_coords.extend(list(zip(y_va_dx[i], y_va_dy[i])))
        
        pred_coords = np.array(pred_coords)
        true_coords = np.array(true_coords)
        
        # (pred_coords - true_coords)**2 -> (N, 2)
        # np.mean(...) -> 计算所有 N*2 个误差的均值
        # np.sqrt(...) -> 求平方根
        rmse = np.sqrt(np.mean((pred_coords - true_coords)**2))
        fold_rmses.append(rmse)
        print(f"\n✓ Fold {fold} - Loss: {loss:.5f}, RMSE: {rmse:.5f}")

    # --- 保存元数据 (在循环结束后, 使用 .pth) ---
    metadata = {
        'input_dim': input_dim,
        'feature_cols': feature_cols, # 保存特征列表
        'window_size': config.WINDOW_SIZE,
        'n_folds': config.N_FOLDS,
    }
    torch.save(metadata, config.OUTPUT_DIR / "metadata.pth")
    print(f"\n✓ Saved metadata (input_dim={input_dim}) to {config.OUTPUT_DIR} (as .pth)")


    # [4/4] CV 摘要
    print("\n[4/4] Calculating CV Summary...")
    all_pred_coords, all_true_coords = [], []
    for sample_id, data in oof_predictions.items():
        all_pred_coords.extend(list(zip(data['pred_dx'], data['pred_dy'])))
        all_true_coords.extend(list(zip(data['true_dx'], data['true_dy'])))
        
    all_pred_coords = np.array(all_pred_coords)
    all_true_coords = np.array(all_true_coords)
    
    # 再次计算总 OOF RMSE
    oof_rmse = np.sqrt(np.mean((all_pred_coords - all_true_coords)**2))
    mean_rmse = np.mean(fold_rmses)
    std_rmse = np.std(fold_rmses)
    
    print("\n" + "="*60)
    print("CV Summary")
    print("="*60)
    print(f"Per-fold RMSEs: [{', '.join([f'{r:.5f}' for r in fold_rmses])}]")
    print(f"每折 RMSE：[{', '.join([f'{r:.5f}' for r in fold_rmses])}]")
    print(f"Mean CV RMSE: {mean_rmse:.4f} ± {std_rmse:.4f}")
    print(f"平均 CV RMSE：{mean_rmse:.4f} ± {std_rmse:.4f}")
    print(f"OOF CV RMSE: {oof_rmse:.4f} (按指标计算的最终 OOF)")
    print("="*60 + "\n")
    
    print("\n" + "🏆"*40)
    print("🏆  TRAINING COMPLETE! 🏆")
    print(f"✓ All models and artifacts saved to {config.OUTPUT_DIR} (as .pth)")
    print("🏆"*40 + "\n")


# ============================================================================
# --- 脚本执行控制器 ---
# ============================================================================

if __name__ == "__main__":
    config = Config()
    
    if config.IS_TRAINING:
        # -----------------
        #  模 式 1: 训 练
        # -----------------
        # 在本地运行或在 Kaggle Notebook 上"保存版本"时执行
        main()
    
    # ----------------------------------------------------------------------
    #  模 式 2: 推 理
    #  当 IS_TRAINING = False 时，`main()` 不会运行，
    #  Python 解释器将继续执行到文件末尾的此 `else` 块和 `predict` 定义
    # ----------------------------------------------------------------------
    else:
        print("[INFERENCE MODE] Script loaded. Defining predict() and starting server...")
        
        # 在 `else` 块中定义 predict 函数，
        # 这样它只在推理模式下存在
        
        def predict(test: pl.DataFrame, test_input: pl.DataFrame) -> pl.DataFrame | pd.DataFrame:
            """
            Inference function: process each batch of data
            (由 Kaggle 推理服务器调用)
            
            Args:
                test: Frames to predict (contains game_id, play_id, nfl_id, frame_id, etc.)
                test_input: Available input data (historical frames)
            
            Returns:
                DataFrame with x, y coordinates
            """
            config = Config()
            
            # --- 1. 首次调用时加载模型 (使用函数属性作为缓存) ---
            if not hasattr(predict, "models_cache"):
                print("[INFERENCE] First call, loading models and artifacts...")
                predict.models_cache = []
                predict.scalers_cache = []
                
                try:
                    # 加载元数据
                    metadata_path = config.OUTPUT_DIR / "metadata.pth"
                    predict.metadata = torch.load(metadata_path, map_location=config.DEVICE,weights_only=False)
                    
                    # 加载路由聚类工件
                    predict.route_kmeans = torch.load(config.OUTPUT_DIR / "route_kmeans.pth", map_location=config.DEVICE,weights_only=False)
                    predict.route_scaler = torch.load(config.OUTPUT_DIR / "route_scaler.pth", map_location=config.DEVICE,weights_only=False)
                    
                    print("✓ Loaded metadata, route K-Means, and route scaler")

                    input_dim = predict.metadata['input_dim']
                    n_folds = predict.metadata['n_folds']
                    print(f"✓ Metadata: input_dim={input_dim}, n_folds={n_folds}")

                    # 加载所有折叠的模型和缩放器
                    for fold in range(1, n_folds + 1):
                        model_path = config.OUTPUT_DIR / f"st_transformer_fold_{fold}.pth"
                        scaler_path = config.OUTPUT_DIR / f"scaler_fold_{fold}.pth"
                        
                        # 加载 scaler
                        scaler = torch.load(scaler_path, map_location=config.DEVICE,weights_only=False)
                        predict.scalers_cache.append(scaler)
                        
                        # 加载 model
                        model = STTransformer(
                            input_dim=input_dim,
                            hidden_dim=config.HIDDEN_DIM,
                            horizon=config.MAX_FUTURE_HORIZON,
                            window_size=config.WINDOW_SIZE,
                            n_heads=config.N_HEADS,
                            n_layers=config.N_LAYERS
                        )
                        model.load_state_dict(torch.load(model_path, map_location=config.DEVICE))
                        model.to(config.DEVICE)
                        model.eval() # 切换到评估模式
                        predict.models_cache.append(model)
                        
                        print(f"✓ Loaded fold {fold} model and scaler")
                    
                    print(f"✓ All {n_folds} models loaded successfully.")
                
                except FileNotFoundError as e:
                    print(f"Error: Missing artifact file! {e}")
                    print("请首先运行训练 (设置 IS_TRAINING = True) 并确保所有 .pth 文件都在 ./outputs/ 中。")
                    # 在服务器环境中，我们应该引发异常来停止
                    raise
            
            # --- 2. 转换为 Pandas ---
            test_pd = test.to_pandas()
            test_input_pd = test_input.to_pandas()
            
            # --- 3. 构建序列 ---
            # (我们从缓存中获取特征工程对象)
            test_seq, test_ids, _, _, feature_cols = prepare_sequences_geometric(
                test_input_pd, test_template=test_pd, is_training=False, 
                window_size=config.WINDOW_SIZE,
                route_kmeans=predict.route_kmeans, 
                route_scaler=predict.route_scaler
            )
            
            if not test_seq:
                # 如果没有生成序列（例如，所有输入都太短）
                print("Warning: No sequences generated for this batch.")
                # 返回一个具有正确形状但为空（或全为0）的 DataFrame
                return pl.DataFrame({'x': np.zeros(len(test_pd)), 'y': np.zeros(len(test_pd))})

            X_test = list(test_seq)
            
            # 获取最后一个 x, y 坐标
            try:
                idx_x = feature_cols.index('x')
                idx_y = feature_cols.index('y')
            except ValueError:
                print("Error: 'x' or 'y' not in feature_cols. Assuming 0 and 1.")
                idx_x, idx_y = 0, 1
                
            x_last = np.array([s[-1, idx_x] for s in X_test], dtype=np.float32)
            y_last = np.array([s[-1, idx_y] for s in X_test], dtype=np.float32)

            # --- 4. 集成预测 ---
            all_preds_ens = []
            for model, sc in zip(predict.models_cache, predict.scalers_cache):
                X_sc = [sc.transform(s) for s in X_test]
                
                fold_preds_batches = []
                with torch.no_grad():
                    for i in range(0, len(X_sc), config.BATCH_SIZE):
                        end = min(i + config.BATCH_SIZE, len(X_sc))
                        batch_x = torch.tensor(np.stack(X_sc[i:end]).astype(np.float32)).to(config.DEVICE)
                        batch_preds = model(batch_x).cpu().numpy()
                        fold_preds_batches.append(batch_preds)
                
                all_preds_ens.append(np.vstack(fold_preds_batches))
            
            # 对所有折叠进行平均
            ens_preds = np.mean(all_preds_ens, axis=0)
            
            # --- 5. 构建与输入 `test` DataFrame 顺序相同的预测 ---
            rows = []
            H = ens_preds.shape[1] # 预测视界 (Horizon)
            
            player_pred_map = {}
            for i, sid in enumerate(test_ids):
                key = (sid['game_id'], sid['play_id'], sid['nfl_id'])
                player_pred_map[key] = {
                    'x_last': x_last[i],
                    'y_last': y_last[i],
                    'preds_dx': ens_preds[i, :, 0],
                    'preds_dy': ens_preds[i, :, 1]
                }
            
            # 计算每个 (gid, pid, nid) 组内的帧序 (t)
            test_pd['t'] = test_pd.groupby(['game_id', 'play_id', 'nfl_id'])['frame_id'].rank(method='first').astype(int) - 1

            for _, row in test_pd.iterrows():
                key = (row['game_id'], row['play_id'], row['nfl_id'])
                t = row['t']
                
                if key in player_pred_map:
                    player_data = player_pred_map[key]
                    tt = min(t, H - 1) # 确保时间步不越界
                    
                    px = np.clip(player_data['x_last'] + player_data['preds_dx'][tt], 0, 120)
                    py = np.clip(player_data['y_last'] + player_data['preds_dy'][tt], 0, 53.3)
                    
                    rows.append({'x': px, 'y': py})
                else:
                    # Fallback
                    # 这可能发生在 prepare_sequences_geometric 丢弃了无效序列时
                    print(f"Warning: Missing prediction for {key} at t={t}. Appending (0, 0).")
                    rows.append({'x': 0.0, 'y': 0.0}) 
                    
            # 转换为 Polars DataFrame
            predictions = pl.DataFrame(rows)
            
            assert len(predictions) == len(test), f"Prediction length ({len(predictions)}) != Test length ({len(test)})"
            
            return predictions

        # --- 服务器启动代码 ---
        # (这部分代码在 IS_TRAINING = False 时才会被执行)
        
        # 初始化推理服务器
        inference_server = kaggle_evaluation.nfl_inference_server.NFLInferenceServer(predict)
        
        # Start server in competition environment
        if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
            print("[SERVER] Starting inference server...")
            inference_server.serve()
        else:
            # 本地测试网关 (如果 KAGGLE_IS_COMPETITION_RERUN 未设置)
            print("[SERVER] Running local gateway for testing...")
            # 确保你的本地数据路径是正确的
            local_test_path = str(config.DATA_DIR)
            if not Path(local_test_path).exists():
                local_test_path = './' # Fallback
                print(f"[SERVER] Warning: DATA_DIR not found, using fallback './'")
            
            print(f"[SERVER] Using data from: {local_test_path}")
            inference_server.run_local_gateway((local_test_path,))